In [ ]:
reeds_scenario = 'ReEDS low renewables price low gas price'

elec_policy_scenario = 'ReEDS'
# options:
# 'ReEDS'
# 'state & local commitments'

In [ ]:
import pandas as pd
import numpy as np
import time

In [ ]:
# input files
gim_cities_file = 'GIM cities in index 2020-10-31.xlsx'
prod_leak_file = 'GIM production area leakage rate by consuming state for 2018 2020-10-31_1106.csv'
trans_leak_state_file = 'GIM average transmission distance for gas by consuming state for analysis year 2018 2020-11-06_0756.csv'

# gim_urban_leak_timestamp = '2020-12-01_0838'
gim_urban_leak_timestamp = '2020-12-01_1652'
gim_urban_leak_file = f'GIM leakage res-comm by urban area compiled {gim_urban_leak_timestamp}.xlsx'

elec_leak_timestamp = '2020-11-30_1653'
elec_leak_file = f'GIM electric sector leakage rates dist & gas meters {elec_leak_timestamp}.csv'

state_abbrev_file = 'US states and abbreviations.xlsx'
parameters_file = 'GIM parameters file.xlsx'

# EIA Electricity Data Browser (EDB)
edb_gas_elec_cons_file = 'EIA EDB NG Consumption_for_electricity_generation_for_electric_power (2020-11-07).csv'
edb_gas_elec_gen_file = 'EIA EDB NG Net_generation_for_electric_power (2020-11-07).csv'

# paths
lca_path = '/Users/masoninman/Dropbox/GEM/LCA of natural gas use/'
inputs_path = lca_path + 'US cities LCA of gas model/US gas model inputs/'
census_path = lca_path + 'US Census/'

In [ ]:
df = pd.read_excel(inputs_path + parameters_file, sheet_name='main parameters')
ser = df.set_index('parameter name')['parameter value']
parameters_main = ser

# read parameters
data_year = parameters_main.at['data_year']
ch4_fract_in_ng_consumer_grade = parameters_main.at['ch4_fract_in_ng_consumer_grade']
ch4_kg_per_mcf = parameters_main.at['ch4_kg_per_mcf']
use_gwp_20y = parameters_main.at['use_gwp_20y']
ch4_gwp_20y = parameters_main.at['ch4_gwp_20y']
ch4_gwp_100y = parameters_main.at['ch4_gwp_100y']
dry_gas_en_dens_btu_per_cf = parameters_main.at['dry_gas_en_dens_btu_per_cf']
emissions_rate_gas_kg_co2_per_mmbtu = parameters_main.at['emissions_rate_gas_kg_co2_per_mmbtu']

em_base_year = 2018
em_eval_start_yr = 2021
em_eval_end_yr = 2036

In [ ]:
# standard conversions
gg_per_kg = 1e-6
gg_per_g = 1e-9
g_per_gg = 1e9
g_per_mt = 1e12
kg_per_g = 1e-3

kwh_per_twh = 1e9
kwh_per_gwh = 1e6
mmbtu_per_btu = 1e-6

mmcf_per_mcf = 1e-3
mcf_per_cf = 1e-3

btu_per_kwh = 3412.14
mmbtu_per_kwh = btu_per_kwh * mmbtu_per_btu

# conversion from volume NG to mass of CH4 it contains
conversion_consumer_ng_mcf_to_ch4_gg = ch4_fract_in_ng_consumer_grade * ch4_kg_per_mcf * gg_per_kg

in each state:

for each kWh electricity consumed:
* calculate how much from NG
  * use ReEDS mean value for fraction of generation from NG in each state, over 2021-2035

for each kWh of electricity from NG:
* calculate how much NG burned (Mcf)
  * use 2019 EIA data to calculate power plant efficiency
  * efficiency = elec consumption of NG / elec gen by NG
  
for each Mcf of NG burned:
* calculate how much leakage occurs
  * use GIM leakage for NG supplied to electricity sector

In [ ]:
def read_state_abbreviations(state_abbrev_file):
    states_df = pd.read_excel(inputs_path + state_abbrev_file)

    states_abbrev_to_full_dict = states_df.set_index('abbrev')['state'].to_dict()
    states_full_to_abbrev_dict = states_df.set_index('state')['abbrev'].to_dict()
    
    return(states_abbrev_to_full_dict, states_full_to_abbrev_dict)

In [ ]:
(states_abbrev_to_full_dict, states_full_to_abbrev_dict) = read_state_abbreviations(state_abbrev_file)

## EIA Electricity Data Browser (EDB)

In [ ]:
df = pd.read_csv(inputs_path + edb_gas_elec_cons_file, header=4)
df = df.loc[df['description']!='Consumption for electricity generation for electric power']
df['state'] = df['description'].str.split(': ').str[1]
df['state'] = df['state'].replace('District Of Columbia', 'District of Columbia')
df['state abbrev'] = df['state'].replace(states_full_to_abbrev_dict)
df = df.rename(columns={'2018': 'NG MMcf consumed elec 2018'})
ser = df.set_index('state abbrev')['NG MMcf consumed elec 2018']
ser = ser.dropna()
ser = ser.replace('--', np.nan)
ser = ser.astype(float)
edb_gas_elec_cons_2018 = ser

df = pd.read_csv(inputs_path + edb_gas_elec_gen_file,header=4)
df = df.loc[df['description']!='Net generation for electric power']
df['state'] = df['description'].str.split(': ').str[1]
df['state'] = df['state'].replace('District Of Columbia', 'District of Columbia')
df['state abbrev'] = df['state'].replace(states_full_to_abbrev_dict)
df = df.rename(columns={'2018': 'NG GWh generated 2018'})
ser = df.set_index('state abbrev')['NG GWh generated 2018']
ser = ser.dropna()
ser = ser.replace('--', np.nan)
ser = ser.astype(float)
edb_gas_elec_gen_2018 = ser

In [ ]:
edb_gas_elec_cons_2018.loc['DC']

In [ ]:
df = pd.concat([edb_gas_elec_cons_2018, edb_gas_elec_gen_2018], axis=1)
conv_factor = kwh_per_gwh * mmcf_per_mcf
df['NG elec efficiency (kWh/Mcf)'] = df['NG GWh generated 2018'].div(df['NG MMcf consumed elec 2018']) * conv_factor
edb_gas_efficiency_2018 = df['NG elec efficiency (kWh/Mcf)']

In [ ]:
edb_gas_efficiency_2018.loc['DC']

## ReEDS scenario
* same as RMI used: low renewables cost, low gas cost
* from https://openei.org/apps/reeds/#

In [ ]:
def calc_reeds_gen_ng_fract_period(reeds_xl):    
    # for each state, calculate mean fraction of electricity from NG 
    # (over period 2021-2035)
    df = pd.read_excel(reeds_xl, sheet_name='genFraction')
    df = df.rename(columns={'value': 'gen fract'})
    df = df.loc[df['tech'].isin(['NG-CC', 'NG-CT'])]
    df = df.groupby(['gid', 'time'])[['gen fract']].sum()

    # unstack states
    df = df.unstack(0)
    df = df.droplevel(0, axis=1)

    # change index to have full range of years
    df = df.reset_index()
    df['time'] = pd.to_datetime(df['time'], format='%Y')
    df = df.set_index('time')
    df = df.resample('Y').agg('mean')

    # interpolate data
    df = df.interpolate()
    df.index = df.index.year

    # calculate average over period 2021-2035
    df = df.loc[df.index.isin(range(2021, 2035+1))]
    ser = df.mean(axis=0)

    ser.name = 'ReEDS elec fract NG'

    reeds_gen_ng_fract_period = ser
    
    return reeds_gen_ng_fract_period

In [ ]:
def calc_reeds_gen_by_state_mean(reeds_xl):
    # for each state, calculate mean annual generation of electricity from all sources 
    # (over period 2021-2035)
    # tested calculations, and got same as shown in map by hovering over; units are TWh
    df = pd.read_excel(reeds_xl, sheet_name='generation')
    df = df.rename(columns={'value': 'generation TWh'})
    # sum over all technologies
    df = df.groupby(['gid', 'time'])[['generation TWh']].sum()

    # unstack states
    df = df.unstack(0)
    df = df.droplevel(0, axis=1)

    # change index to have full range of years
    df = df.reset_index()
    df['time'] = pd.to_datetime(df['time'], format='%Y')
    df = df.set_index('time')
    df = df.resample('Y').agg('mean')

    # interpolate data
    df = df.interpolate()
    df.index = df.index.year

    # calculate average over period 2021-2035
    df = df.loc[df.index.isin(range(2021, 2035+1))]
    ser = df.mean(axis=0)

    ser.name = 'ReEDS elec all TWh/y'

    reeds_gen_all_mean = ser
    
    return reeds_gen_all_mean

In [ ]:
def calc_reeds_elec_co2_period(reeds_xl):
    """  
    For each state, calculate CO2 emissions from all electricity (over period 2021-2035)
    
    Units: billion metric tons CO2
    Based on viewer (https://openei.org/apps/reeds), which had values matching my calculations.
    """
    df = pd.read_excel(reeds_xl, sheet_name='co2')
    df = df.rename(columns={'value': 'CO2 Gtons'})
    df = df.groupby(['gid', 'time'])[['CO2 Gtons']].sum()

    # unstack states
    df = df.unstack(0)
    df = df.droplevel(0, axis=1)

    # change index to have full range of years
    df = df.reset_index()
    df['time'] = pd.to_datetime(df['time'], format='%Y')
    df = df.set_index('time')
    df = df.resample('Y').agg('mean')

    # interpolate data
    df = df.interpolate()
    df.index = df.index.year

    # calculate average over period 2021-2035
    df = df.loc[df.index.isin(range(2021, 2035+1))]
    ser = df.mean(axis=0)

    # convert to million tons
    mtons_per_gton = 1e3
    ser = ser * mtons_per_gton
    ser.name = 'ReEDS elec CO2 (Mton/y)'

    reeds_elec_co2_period = ser
    
    return reeds_elec_co2_period

In [ ]:
def calc_reeds_elec_co2_traj(reeds_xl):
    """  
    For each state, calculate CO2 emissions from all electricity (over period 2021-2035)
    
    Units: billion metric tons CO2
    Based on viewer (https://openei.org/apps/reeds), which had values matching my calculations.
    """
    df = pd.read_excel(reeds_xl, sheet_name='co2')
    df = df.rename(columns={'value': 'CO2 Gtons'})
    df = df.groupby(['gid', 'time'])[['CO2 Gtons']].sum()

    # unstack states
    df = df.unstack(0)
    df = df.droplevel(0, axis=1)

    # change index to have full range of years
    df = df.reset_index()
    df['time'] = pd.to_datetime(df['time'], format='%Y')
    df = df.set_index('time')
    df = df.resample('Y').agg('mean')

    # interpolate data
    df = df.interpolate()
    df.index = df.index.year

    # convert to million tons
    mtons_per_gton = 1e3
    for col in df.columns:
        df[col] = df[col] * mtons_per_gton
        
    reeds_elec_co2_mt_per_y_traj = df
    
    return reeds_elec_co2_mt_per_y_traj

In [ ]:
def calc_reeds_gen_by_state_traj(reeds_xl):
    # for each state, calculate mean annual generation of electricity from all sources 
    # (over period 2021-2035)
    # tested calculations, and got same as shown in map by hovering over; units are TWh
    df = pd.read_excel(reeds_xl, sheet_name='generation')
    df = df.rename(columns={'value': 'generation TWh'})
    # sum over all technologies
    df = df.groupby(['gid', 'time'])[['generation TWh']].sum()

    # unstack states
    df = df.unstack(0)
    df = df.droplevel(0, axis=1)

    # change index to have full range of years
    df = df.reset_index()
    df['time'] = pd.to_datetime(df['time'], format='%Y')
    df = df.set_index('time')
    df = df.resample('Y').agg('mean')

    # interpolate data
    df = df.interpolate()
    df.index = df.index.year

    reeds_gen_by_state_traj = df
    
    return reeds_gen_by_state_traj

In [ ]:
def calc_reeds_gen_by_state_2018(reeds_xl):
    # for each state, calculate mean annual generation of electricity from all sources in 2018
    # tested calculations, and got same as shown in map by hovering over; units are TWh
    df = pd.read_excel(reeds_xl, sheet_name='generation')
    df = df.rename(columns={'value': 'generation TWh'})
    # sum over all technologies
    df = df.groupby(['gid', 'time'])[['generation TWh']].sum()

    # unstack states
    df = df.unstack(0)
    df = df.droplevel(0, axis=1)

    # get value in 2018
    df = df.loc[df.index==2018]
    ser = df.mean(axis=0)

    ser.name = 'ReEDS elec all TWh/y'

    reeds_gen_all_2018 = ser

    return reeds_gen_all_2018

In [ ]:
def calc_reeds_elec_co2_2018(reeds_xl):
    """  
    For each state, calculate CO2 emissions from all electricity (over period 2021-2035)
    
    Units: billion metric tons CO2
    Based on viewer (https://openei.org/apps/reeds), which had values matching my calculations.
    """
    df = pd.read_excel(reeds_xl, sheet_name='co2')
    df = df.rename(columns={'value': 'CO2 Gtons'})
    df = df.groupby(['gid', 'time'])[['CO2 Gtons']].sum()

    # unstack states
    df = df.unstack(0)
    df = df.droplevel(0, axis=1)

    # get values for 2018
    df = df.loc[df.index==2018]
    ser = df.mean(axis=0)

    # convert to million tons
    mtons_per_gton = 1e3
    ser = ser * mtons_per_gton
    ser.name = 'ReEDS elec CO2 (Mton/y)'

    reeds_elec_co2_2018 = ser
    
    return reeds_elec_co2_2018

In [ ]:
def calc_reeds_ng_fract_2018(reeds_xl):    
    # for each state, calculate fraction of electricity from NG in 2018
    df = pd.read_excel(reeds_xl, sheet_name='genFraction')
    df = df.rename(columns={'value': 'gen fract'})
    df = df.loc[df['tech'].isin(['NG-CC', 'NG-CT'])]
    df = df.groupby(['gid', 'time'])[['gen fract']].sum()

    # unstack states
    df = df.unstack(0)
    df = df.droplevel(0, axis=1)

    # get value for 2018
    df = df.loc[df.index==2018]
    ser = df.mean(axis=0)

    ser.name = 'ReEDS elec fract NG'

    reeds_gen_ng_fract_2018 = ser
    
    return reeds_gen_ng_fract_2018

In [ ]:
# create ReEDS results for period (2021-2035)

# read ReEDS scenario file
if reeds_scenario == 'ReEDS low renewables price low gas price':
    reeds_file = 'NREL ReEDS 2019 - LowRECostLowNGPrice.xlsx'
    reeds_xl = pd.ExcelFile(inputs_path + reeds_file)
else:
    print("ERROR!" + f" No file for specified ReEDS scenario.")

reeds_gen_ng_fract_period = calc_reeds_gen_ng_fract_period(reeds_xl)
reeds_gen_all_mean = calc_reeds_gen_by_state_mean(reeds_xl)
reeds_elec_co2_period = calc_reeds_elec_co2_period(reeds_xl)

reeds_all_period = pd.concat([
    reeds_gen_all_mean, 
    reeds_gen_ng_fract_period, 
    reeds_elec_co2_period,
], axis=1)

reeds_all_period['ReEDS elec NG TWh/y'] = reeds_all_period[['ReEDS elec all TWh/y', 'ReEDS elec fract NG']].product(axis=1)

In [ ]:
# create ReEDS results for 2018

# read ReEDS scenario files
if reeds_scenario == 'ReEDS low renewables price low gas price':
    reeds_file = 'NREL ReEDS 2019 - LowRECostLowNGPrice.xlsx'
    reeds_xl = pd.ExcelFile(inputs_path + reeds_file)
else:
    print("ERROR!" + f" No file for specified ReEDS scenario.")

reeds_gen_ng_fract_2018 = calc_reeds_ng_fract_2018(reeds_xl)
reeds_gen_all_2018 = calc_reeds_gen_by_state_2018(reeds_xl)
reeds_elec_co2_2018 = calc_reeds_elec_co2_2018(reeds_xl)
reeds_all_2018 = pd.concat([
    reeds_gen_all_2018, 
    reeds_gen_ng_fract_2018, 
    reeds_elec_co2_2018,
], axis=1)

reeds_all_2018['ReEDS elec NG TWh/y'] = reeds_all_2018[['ReEDS elec all TWh/y', 'ReEDS elec fract NG']].product(axis=1)

In [ ]:
reeds_elec_co2_mt_per_y_traj = calc_reeds_elec_co2_traj(reeds_xl)
reeds_gen_by_state_traj = calc_reeds_gen_by_state_traj(reeds_xl)

In [ ]:
# show:
# for each state, get fraction of electricity generated in state, from each source
df = pd.read_excel(reeds_xl, sheet_name='genFraction')
df = df.rename(columns={'value': 'genFract'})

sel_state = 'CA'
df = df.loc[df['gid']==sel_state]
df = df.set_index(['gid', 'time', 'tech']).unstack(-1)
df.columns = df.columns.droplevel(0)
df['clean fract'] = df[[
    'Biopower', 'CSP', 'Geothermal', 'Hydro', 
    'Land-based Wind', 'Nuclear', 'Offshore Wind', 
    'Rooftop PV', 'Utility PV']].sum(axis=1)
df['renew fract'] = df[[
    'Biopower', 'CSP', 'Geothermal', 'Hydro', 
    'Land-based Wind', 'Offshore Wind', 
    'Rooftop PV', 'Utility PV']].sum(axis=1)
df

## GIM leakage rate for NG supply to electric sector

### pull prod, trans, & elec sector leakage from GIM LDC leakage notebook
include distribution pipeline leakage assigned to electric sector
* each gas company is assigned to a specific state (whether LDC or non-LDC)
* we know gas deliveries to electric sector by gas company 
* for each gas company, calculate quantity of leakage (prod, trans, distribution) for gas deliveries to electric sector
* sum values across gas companies to get a quantity of leakage for each state
* from EIA, we know generation from gas for each state
* calculate leakage rate (g CH4 leaked/kWh electricity generated) for gas for electricity for each state

In [ ]:
def create_elec_leak_by_state():
    # Production area leakage results, by consuming state:
    prod_leak = pd.read_csv(lca_path + prod_leak_file)
    if prod_leak.columns[0]=='Unnamed: 0':
        prod_leak = prod_leak.rename(columns={'Unnamed: 0': 'state'})
    prod_leak['state abbrev'] = prod_leak['state'].replace(states_full_to_abbrev_dict)
    prod_leak = prod_leak.loc[~prod_leak['state'].isin(['Canada', 'Mexico'])]

    # Transmission area leakage results
    trans_leak_state = pd.read_csv(lca_path + trans_leak_state_file)
    trans_leak_state['state abbrev'] = trans_leak_state['state'].replace(states_full_to_abbrev_dict)

    # create df of leakage rates for gas supplied to electric sector, drawing on:
    # production leakage by state
    # transmission leakage by state
    # distribution pipeline leakage by state
    # electric sector meter leakage fraction by state

    df = pd.merge(
        prod_leak[['state abbrev', 'prod leak g CH4/Mcf dry gas']], 
        trans_leak_state[['state abbrev', 'trans leak g CH4/Mcf']], 
        on='state abbrev', how='outer')
    
    # -----------------
    # electric distribution pipeline leakage
#     dist_pipe_elec_leak_g_ch4_per_mcf_by_state = pd.read_csv(
#         lca_path + f'GIM distribution pipeline leakage electric sector by state {dist_pipe_elec_leak_g_ch4_per_mcf_by_state_timestamp}.csv')

#     elec_leak = pd.read_csv(f'GIM electric sector leakage rates dist & gas meters {elec_leak_timestamp}.csv')

#     dist_pipe_elec_leak_g_ch4_per_mcf_by_state = 
#     dist_pipe_elec_leak_ser = dist_pipe_elec_leak_g_ch4_per_mcf_by_state.set_index('state')['dist pipe elec leak g CH4/Mcf']
#     dist_pipe_elec_leak_dict = dist_pipe_elec_leak_ser.to_dict()
    
#     # put values into df
#     df['dist pipe elec g CH4/Mcf'] = df['state abbrev'].replace(dist_pipe_elec_leak_dict)
    
#     # -----------------
#     # electric customer gas meter leakage
#     elec_meter_leak_fract = pd.read_csv(lca_path + f'GIM elec_meter_leak_fract {elec_meter_leak_fract_timestamp}.csv')
#     elec_meter_leak_fract = elec_meter_leak_fract.set_index('State')['elec meter leak fract']
#     elec_cust_meter_leak_g_ch4_per_mcf = elec_meter_leak_fract * conversion_consumer_ng_mcf_to_ch4_gg * g_per_gg
#     elec_cust_meter_leak_g_ch4_per_mcf_dict = elec_cust_meter_leak_g_ch4_per_mcf.to_dict()
    
#     df['cust meter elec leak g CH4/Mcf'] = df['state abbrev'].replace(elec_cust_meter_leak_g_ch4_per_mcf_dict)

    elec_leak = pd.read_csv(
        inputs_path + elec_leak_file)
    elec_leak = elec_leak.rename(columns={'Unnamed: 0': 'state abbrev'})
    
    # merge in
    df = pd.merge(df, elec_leak, on='state abbrev', how='left')

    # -----------------
    # calculate sum of all components
    df['elec leak total g CH4/Mcf'] = df[[
        'prod leak g CH4/Mcf dry gas', 
        'trans leak g CH4/Mcf', 
        'dist pipe elec leak g CH4/Mcf',
        'cust meter elec leak g CH4/Mcf',
    ]].sum(axis=1)

    df = df.set_index('state abbrev')
    
    elec_leak_by_state = df
    return elec_leak_by_state

In [ ]:
elec_leak_by_state = create_elec_leak_by_state()

In [ ]:
elec_leak_by_state.head()

## Combine all results

In [ ]:
def create_elec_em(edb_gas_efficiency_2018, reeds_all_period, elec_leak_by_state):
    # merge edb & reeds
    df = pd.merge(
        edb_gas_efficiency_2018,
        reeds_all_period,
        left_index=True,
        right_index=True,
    )
    # merge in GIM leakage rate
    df = pd.merge(
        df[['ReEDS elec all TWh/y', 'ReEDS elec CO2 (Mton/y)', 'ReEDS elec NG TWh/y', 'NG elec efficiency (kWh/Mcf)']],
        elec_leak_by_state['elec leak total g CH4/Mcf'],
        left_index=True,
        right_index=True
    )

    # calculate quantity of gas consumed for electricity
    mcf_gas_for_elec = (df['ReEDS elec NG TWh/y'] * kwh_per_twh) / df['NG elec efficiency (kWh/Mcf)']

    # calculate quantity of CH4 leaked from gas for electricity
    df['elec leak CH4 Gg/y'] = mcf_gas_for_elec * df['elec leak total g CH4/Mcf'] * gg_per_g

    # assign regions, roughly following RMI:
    df['region'] = df.index
    for state in df.index:
        if state in ['CT', 'MA', 'ME', 'NH', 'RI', 'VT']:
            df.at[state, 'region'] = 'ISO NE'
            
        elif state in ['AR', 'IA', 'IL', 'IN', 'LA', 'MI', 'MN', 'MO', 'MS', 'ND', 'WI']:
            df.at[state, 'region'] = 'MISO'
            
        elif state in ['DE', 'DC', 'MD', 'NJ', 'OH', 'PA', 'VA', 'WV']:
            df.at[state, 'region'] = 'PJM'
            
        elif state in ['KS', 'KY', 'NE', 'OK', 'SD']:
            df.at[state, 'region'] = 'SPP'
        
        else:
            # don't change; leave as individual state
            pass

    # calculate state/regional values
    regional = df.groupby('region')[[
        'ReEDS elec all TWh/y', 'ReEDS elec CO2 (Mton/y)',
        'ReEDS elec NG TWh/y', 'elec leak CH4 Gg/y']].sum()

    regional['g CH4 leak/kWh gas elec'] = (regional['elec leak CH4 Gg/y'] * g_per_gg) / (regional['ReEDS elec NG TWh/y'] * kwh_per_twh)
    regional['g CH4 leak/kWh all elec'] = (regional['elec leak CH4 Gg/y'] * g_per_gg) / (regional['ReEDS elec all TWh/y'] * kwh_per_twh)

    regional['g CO2/kWh elec'] = (regional['ReEDS elec CO2 (Mton/y)'] * g_per_mt) / (regional['ReEDS elec all TWh/y'] * kwh_per_twh)

    # specify CH4 GWP; calculate total emissions
    use_gwp_20y = parameters_main.at['use_gwp_20y']
    
    if use_gwp_20y == True:
        regional['CH4 GWP (20-y)'] = parameters_main.at['ch4_gwp_20y']
        print(f"use_gwp_20y is True; CH4 20-y GWP value: {parameters_main.at['ch4_gwp_20y']}") # for db
        regional['total g CO2e/kWh elec'] = regional['g CO2/kWh elec'] + regional[['g CH4 leak/kWh all elec', 'CH4 GWP (20-y)']].product(axis=1)
    else:
        regional['CH4 GWP (100-y)'] = parameters_main.at['ch4_gwp_100y']
        print(f"use_gwp_20y is not True; CH4 100-year GWP value: {parameters_main.at['ch4_gwp_100y']}") # for db
        regional['total g CO2e/kWh elec'] = regional['g CO2/kWh elec'] + regional[['g CH4 leak/kWh all elec', 'CH4 GWP (100-y)']].product(axis=1)
    regional = regional.reset_index()
    
    # merge back into state df
    df = pd.merge(
        df.reset_index().rename(columns={'index': 'state'}),
        regional[['region', 'g CO2/kWh elec', 'total g CO2e/kWh elec']],
        on='region', how='outer',        
    )

    elec_em_period = df
    return elec_em_period

In [ ]:
elec_em_2018 = create_elec_em(edb_gas_efficiency_2018, reeds_all_2018, elec_leak_by_state)
elec_em_period = create_elec_em(edb_gas_efficiency_2018, reeds_all_period, elec_leak_by_state)

In [ ]:
elec_em_period.loc[elec_em_period['state'].isin(['SC', 'MD', 'VA', 'WV', 'NC', 'DE', 'GA'])]

In [ ]:
# # Note: this doesn't have DC yet; that's added below
# compare = pd.merge(
#     elec_em_2018[['state', 'g CO2/kWh elec']],
#     elec_em_period[['state', 'g CO2/kWh elec']],
#     on='state', suffixes=[' 2018', ' projection'])
# compare['fract reduction'] = (compare['g CO2/kWh elec 2018'] - compare['g CO2/kWh elec projection'])/compare['g CO2/kWh elec 2018']
# compare = compare.sort_values(by='fract reduction')
# compare

In [ ]:
# Docs: Table 15-1
docs_table_15_1 = elec_em_period.set_index('region').sort_index()[
    ['g CO2/kWh elec', 'total g CO2e/kWh elec']].drop_duplicates()

docs_table_15_1

## Assign emissions rate for each urban area

In [ ]:
def create_elec_em_for_urban_area_based_on_state(elec_em_period):
    """
    Create df with emissions for each urban area, based on states. 
    """
    df = pd.read_excel(inputs_path + gim_cities_file)
    df = df.rename(columns={'Census urban area name': 'urban name'})
    df['metro st'] = df['metro area'] + ', ' + df['metro area state']

    df = df.drop(['UACE', 'AFFGEOID', 'GEOID', 'notes'], axis=1)
    urban_state = df
    
    elec_em_period_urb = pd.merge(
        urban_state[['metro st', 'urban name', 'metro area state']],
        elec_em_period[['state', 'total g CO2e/kWh elec']],
        left_on = 'metro area state', right_on='state',
        how='outer'
    )
    
    # fill in value for DC (from PJM)
    dc_index = elec_em_period_urb.loc[elec_em_period_urb['metro st']=='District of Columbia, DC'].index[0]
    elec_em_period_urb.at[dc_index, 'state'] = 'DC'
    pjm_em = elec_em_period.set_index('region').at['PJM', 'g CO2/kWh elec'].mean()
    elec_em_period_urb.at[dc_index, 'total g CO2e/kWh elec'] = pjm_em

    # modify Burlington, VT & Seattle, WA to be zero emissions 
    # (already achieved as of 2018)
    burl_seat_indices = elec_em_period_urb.loc[elec_em_period_urb['metro st'].isin(['Burlington, VT', 'Seattle, WA'])].index
    for row in burl_seat_indices:
        elec_em_period_urb.at[row, 'total g CO2e/kWh elec'] = 0
        
    # drop rows with no urban name (for DE & NJ)
    elec_em_period_urb = elec_em_period_urb.dropna(subset=['urban name'])
    
    return elec_em_period_urb

In [ ]:
# new version of function
def create_electrification_df(gas_elec_em_period_urb):
    df = gas_elec_em_period_urb
    
    # import production area & transmission leakage
    
    # import components of leakage from previous notebook
    urban_leak = pd.read_excel(lca_path + gim_urban_leak_file)
    urban_leak = urban_leak.rename(columns={
        'total rc leak g CH4/Mcf': 'life cycle total rc leak g CH4/Mcf'})
    
    df = pd.merge(
        df,
        urban_leak[['urban name', 'life cycle total rc leak g CH4/Mcf']],
        on='urban name',
        how='outer',
    )

    # import and merge RMI data
    df['state abbrev'] = df['urban name'].str.split(', ').str[-1].str.split('--').str[0]

    rmi_file = 'RMI building electrification - heat pumps (June 2020).xlsx'
    rmi = pd.read_excel(lca_path + rmi_file, sheet_name='original')
    rmi = rmi.rename(columns={
        'State': 'state abbrev',
        'Heating season-weighted COP': 'COP',
    })
    rmi_cop = rmi[['state abbrev', 'Gas AFUE', 'COP']]
    df = pd.merge(df, rmi_cop, on='state abbrev', how='outer')
    df = df.drop('state abbrev', axis=1)
    
    # calculate emissions per mmbtu    
    df['conv elec heat: kg CO2e/mmbtu'] = df['total g CO2e/kWh elec'] * kg_per_g / mmbtu_per_kwh

    df['NG: kg CO2/mmbtu'] = emissions_rate_gas_kg_co2_per_mmbtu
    
    # calculate emissions per mmbtu of heat delivered
    df['total rc leak g CH4/mmbtu'] = df['life cycle total rc leak g CH4/Mcf'] * mcf_per_cf / (dry_gas_en_dens_btu_per_cf * mmbtu_per_btu)

    if use_gwp_20y == True:
        df['CH4 GWP (20-y)'] = parameters_main.at['ch4_gwp_20y']
#         print(f"use_gwp_20y is True; CH4 20-y GWP value: {parameters_main.at['ch4_gwp_20y']}") # for db
        df['NG leak: kg CO2e/mmbtu gas'] = df['total rc leak g CH4/mmbtu'] * df['CH4 GWP (20-y)'] * kg_per_g
    else:
        df['CH4 GWP (100-y)'] = parameters_main.at['ch4_gwp_100y']
#         print(f"use_gwp_20y is not True; CH4 100-year GWP value: {parameters_main.at['ch4_gwp_100y']}") # for db
        df['NG leak: kg CO2e/mmbtu gas'] = df['total rc leak g CH4/mmbtu'] * df['CH4 GWP (100-y)'] * kg_per_g

    df['NG: total kg CO2e/mmbtu'] = df['NG: kg CO2/mmbtu'] + df['NG leak: kg CO2e/mmbtu gas']

    df['NG: kg CO2e/mmbtu heat'] = df['NG: total kg CO2e/mmbtu'] / df['Gas AFUE']

    df['em savings % conv elec'] = (df['NG: kg CO2e/mmbtu heat'] - df['conv elec heat: kg CO2e/mmbtu'])/df['NG: kg CO2e/mmbtu heat']

    df = df.sort_values(by='em savings % conv elec')

    df['elec heat pump: kg CO2e/mmbtu'] = df['conv elec heat: kg CO2e/mmbtu'] / df['COP']
    df['em savings % heat pump'] = (df['NG: kg CO2e/mmbtu heat'] - df['elec heat pump: kg CO2e/mmbtu']) / df['NG: kg CO2e/mmbtu heat']

    df = df.dropna(subset=['urban name'])
    df = df.set_index('urban name')
    
    elec_em_period_savings = df
    return elec_em_period_savings

In [ ]:
def modify_elec_traj_city_state_policies(
    gas_elec_em_period_urb, 
    gas_elec_em_2018):
    
    # get commitments from GIM cities file
    df = pd.read_excel(inputs_path + gim_cities_file)
    df = df.rename(columns={'Census urban area name': 'urban name'})
    df['metro st'] = df['metro area'] + ', ' + df['metro area state']
    df = df.set_index('urban name')
    df = df.drop(['UACE', 'AFFGEOID', 'GEOID', 'notes'], axis=1)
    commits = df[['clean elec commit %', 'clean elec commit year']]

    # create df of emissions projection, which is modified below based on policies
    df_em = gas_elec_em_period_urb.copy()
    df_em = df_em.set_index('urban name')
    
    df_em_mod = df_em.copy()

    for urban in df_em_mod.index:
        print(f"processing {urban}")
        if urban in ['Burlington, VT', 'Seattle, WA']:
            # these cities' electricity was already zero-emissions (or net-zero emissions) in 2018
            # don't change
            pass
            
        else:
            commit_pct = df.at[urban, 'clean elec commit %']
            if commit_pct > 0:
                commit_yr = commits.at[urban, 'clean elec commit year']

                # get 2018 emissions rate
                elec_em_rate_2018 = df_em_mod.at[urban, 'total g CO2e/kWh elec']

                # 100% reduction targets
                if commit_yr > em_eval_start_yr:

                    em_traj_slope = (0 - elec_em_rate_2018) / (commit_yr - em_base_year)
                    em_traj_intercept = commit_yr * elec_em_rate_2018 / (commit_yr - em_base_year)

                    # calculate total emissions over period 2021-2036, then calculate annual average
                    # emissions function:
                    em_over_period = 0 # initialize
                    for year in range(em_eval_start_yr, em_eval_end_yr+1):
                        em_year = (em_traj_slope * year) + em_traj_intercept
                        if em_year < 0:
                            em_year = 0
                        em_over_period += em_year

                    number_of_yr = em_eval_end_yr - (em_eval_start_yr - 1)
                    em_over_period_ann_avg = em_over_period / number_of_yr

                    df_em_mod.at[urban, 'total g CO2e/kWh elec'] = em_over_period_ann_avg
                    df_em_mod.at[urban, 'status'] = 'modified'

                elif commit_yr <= em_eval_start_yr:
                    # already met target of 100% clean
                    df_em_mod.at[urban, 'total g CO2e/kWh elec'] = 0
                    df_em_mod.at[urban, 'status'] = 'modified'

            else:
                # don't modify values in df
                df_em_mod.at[urban, 'status'] = 'unmodified'
            
    df_em_mod = df_em_mod.reset_index()
    
    gas_elec_em_period_avg_urb_mod = df_em_mod
    return gas_elec_em_period_avg_urb_mod

In [ ]:
# to modify the results for city/state policies for clean electricity,
# need to get the emissions rate in 2018
# then can ramp down emissions
elec_em_2018_urb = create_elec_em_for_urban_area_based_on_state(elec_em_2018)

In [ ]:
# get emissions over the specified period (lifetime of appliances)
elec_em_period_urb = create_elec_em_for_urban_area_based_on_state(elec_em_period)

if elec_policy_scenario == 'state & local commitments':
    # run function to modify gas_elec_em_period_avg
    print("running modify_elec_traj_city_state_policies") # for db
    elec_em_period_urb_to_process = modify_elec_traj_city_state_policies(elec_em_period_urb, elec_em_2018_urb)
else:
    elec_em_period_urb_to_process = elec_em_period_urb
    
elec_em_period_urb_savings = create_electrification_df(elec_em_period_urb_to_process)

In [ ]:
# what the input spreadsheet did:
# if a state is committed to 100% clean electricity by year Y, use that for all cities in the state
# if a city is committed to 100% clean electricity by year < Y, use that instead for that city

# so take the year and commitment % from the spreadsheet:
# if there is an entry for a given city,
# calculate ramp down of emissions rate from ReEDS 2018 rate to year Y
# then assume same emissions rate for years > Y

# calculate average emissions rate over period 2021-2035

# to calculate the share of NG over that period:
# take share of NG in 2020, and assume linear ramp-down to zero in year X
# then calculate average share of NG over 2021-2035

### merge in coordinates

In [ ]:
df = pd.read_excel(inputs_path + gim_cities_file)
df = df.rename(columns={'Census urban area name': 'urban name'})
df = df.set_index('urban name')[['latitude', 'longitude']]
city_coords = df

In [ ]:
df = pd.merge(elec_em_period_urb_savings, city_coords, left_index=True, right_index=True, how='outer')
df = df.reset_index()

if elec_policy_scenario == 'state & local commitments':
    scenario_short = 'commits'
    if 'status' in df.columns:
        df = df.drop('status', axis=1)
        
elif elec_policy_scenario == 'ReEDS':
    scenario_short = 'ReEDS'
else:
    print("ERROR!" + f" Specified scenario wasn't listed above: {elec_policy_scenario}")

df.insert(4, 'scenario', scenario_short)
elec_em_period_urb_savings_coords = df

In [ ]:
if use_gwp_20y == True:
    gwp_for_file = 'GWP20'
else:
    gwp_for_file = 'GWP100'

# export electricity results
save_timestamp = time.strftime('%Y-%m-%d_%H%M', time.localtime())
elec_em_period_urb_savings_coords.to_excel(
    lca_path + f'GIM electrification emissions - {scenario_short} {gwp_for_file} {save_timestamp}.xlsx', 
    index=False)

# create tables for specific maps for website:
# conventional electric furnace map
scenario_conv_elec = elec_em_period_urb_savings_coords[[
    'scenario', 'metro st', 
    'latitude', 'longitude', 
    'em savings % conv elec']]
scenario_conv_elec.insert(0, 'timestamp', save_timestamp)
scenario_conv_elec.insert(2, 'tech', 'conv elec')
scenario_conv_elec = scenario_conv_elec.rename(columns={
    'em savings % conv elec': 'Emissions savings'})
scenario_conv_elec.to_csv(
    lca_path + f'GIM electrification emissions for map - {scenario_short} conv elec {gwp_for_file} {save_timestamp}.csv', 
    index=False)

# electric heat pump map
scenario_heat_pump = elec_em_period_urb_savings_coords[[
    'scenario', 'metro st', 
    'latitude', 'longitude', 
    'em savings % heat pump']]
scenario_heat_pump.insert(0, 'timestamp', save_timestamp)
scenario_heat_pump.insert(2, 'tech', 'heat pump')
scenario_heat_pump = scenario_heat_pump.rename(columns={
    'em savings % heat pump': 'Emissions savings'})
scenario_heat_pump.to_csv(
    lca_path + f'GIM electrification emissions for map - {scenario_short} heat pump {gwp_for_file} {save_timestamp}.csv', 
    index=False)

# end of module